In [2]:
import time

import IPython.display
import ipywidgets
import openai


input = ipywidgets.widgets.Textarea(
    value="""Well now, partner, you've made it to Tech Support. Let's see if we can't get you fixed up.\n\nIf your internet's givin' you trouble, press 1, and we'll get it back in line. Need help with billing or account details? Press 2, and we'll sort it out.\n\nFor device setup or software issues, press 3, and we'll walk you through, nice and easy. If it's somethin' urgent, press 0, and we'll get you talkin' to a real person.\n\nNo need to fret, we'll have you back in the saddle in no time.""",
    description="Input: ",
    layout=ipywidgets.Layout(width="auto"),
)

instructions = ipywidgets.widgets.Textarea(
    value="""Voice: Warm, relaxed, and friendly, with a steady cowboy drawl that feels approachable.\n\nPunctuation: Light and natural, with gentle pauses that create a conversational rhythm without feeling rushed.\n\nDelivery: Smooth and easygoing, with a laid-back pace that reassures the listener while keeping things clear.\n\nPhrasing: Simple, direct, and folksy, using casual, familiar language to make technical support feel more personable.\n\nTone: Lighthearted and welcoming, with a calm confidence that puts the caller at ease.""",
    description="Instructions: ",
)

voice = ipywidgets.widgets.Dropdown(
    options=["Alloy", "Ash", "Ballad", "Coral", "Echo", "Fable", "Onyx", "Nova", "Sage", "Shimmer", "Verse"],
    value="Coral",
    description="Voice: ",
)

response_format = ipywidgets.widgets.Dropdown(
    options=["mp3", "mp4", "mpeg", "mpga", "m4a", "wav", "webm"],
    value="wav",
    description="Response Format: ",
)

output = ipywidgets.widgets.Output()

button = ipywidgets.widgets.Button(
    description="Generate",
)


@output.capture()
def generate(b: ipywidgets.widgets.Button):
    b.disabled = True

    filename = f"/tmp/{time.time()}.{response_format.value}"

    client = openai.OpenAI(
        base_url="http://cortex-api.cortex-api.svc.cluster.local:8080/v1",
        api_key=open("/var/run/secrets/kubernetes.io/serviceaccount/token", "r", encoding="utf-8").read(),
    )

    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice=voice.value.lower(),
        input=input.value,
        instructions=instructions.value,
        response_format=response_format.value,
    ) as response:
        response.stream_to_file(filename)

    b.disabled = False

    IPython.display.display(IPython.display.Audio(filename))


button.on_click(generate)

IPython.display.display(input, instructions, voice, response_format, button, output)

Text(value="Well now, partner, you've made it to Tech Support. Let's see if we can't get you fixed up.\n\nIf y…

Text(value='Voice: Warm, relaxed, and friendly, with a steady cowboy drawl that feels approachable.\n\nPunctua…

Dropdown(description='Voice: ', index=3, options=('Alloy', 'Ash', 'Ballad', 'Coral', 'Echo', 'Fable', 'Onyx', …

Dropdown(description='Response Format: ', index=5, options=('mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm'…

Button(description='Generate', style=ButtonStyle())

Output()